In [ ]:
import os
import json
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tqdm import tqdm

def load_images_and_masks(relation_folder, image_dir, mask_dir, augmented_dir, target_size=(128, 128), num_classes=21):
    collected_image_names = set()

    json_files = [f for f in os.listdir(relation_folder) if f.endswith('.json')]
    for json_file in tqdm(json_files, desc="Collecting image names from JSON files"):
        json_path = os.path.join(relation_folder, json_file)
        with open(json_path, 'r') as f:
            data = json.load(f)
            file_names = data.get("fileNames", [])
            for file_name in file_names:
                base_name = os.path.splitext(file_name)[0]
                collected_image_names.add(base_name)
    
    images = []
    masks = []
    
    for base_name in tqdm(sorted(collected_image_names), desc="Loading images and masks from relations"):
        img_path = os.path.join(image_dir, f"{base_name}.jpg")
        mask_path = os.path.join(mask_dir, f"{base_name}.png")
        
        if os.path.exists(img_path):
            img = load_img(img_path, target_size=target_size)
            img = img_to_array(img) / 255.0
            images.append(img)
        
        if os.path.exists(mask_path):
            mask = load_img(mask_path, target_size=target_size, color_mode='grayscale')
            mask = img_to_array(mask).astype(np.int32)
            mask = np.squeeze(mask, axis=-1)
            
            mask[mask > 20] = 0
            
            mask = tf.keras.utils.to_categorical(mask, num_classes=num_classes)
            masks.append(mask)
    
    augmented_files = os.listdir(augmented_dir)
    augmented_base_names = {f.split('-')[0] for f in augmented_files}
    
    for base_name in tqdm(sorted(augmented_base_names), desc="Loading augmented images and masks"):
        img_files = [f for f in augmented_files if f.startswith(base_name)]
        for img_file in img_files:
            aug_img_path = os.path.join(augmented_dir, img_file)
            mask_path = os.path.join(mask_dir, f"{base_name}.png")
            
            if os.path.exists(aug_img_path):
                img = load_img(aug_img_path, target_size=target_size)
                img = img_to_array(img) / 255.0
                images.append(img)
            
            if os.path.exists(mask_path):
                mask = load_img(mask_path, target_size=target_size, color_mode='grayscale')
                mask = img_to_array(mask).astype(np.int32)
                mask = np.squeeze(mask, axis=-1)
                
                mask[mask > 20] = 0
                
                mask = tf.keras.utils.to_categorical(mask, num_classes=num_classes)
                masks.append(mask)
    
    return np.array(images), np.array(masks)

relation_folder = '../data/relations'
image_dir = '../data/VOCdevkit/VOC2012/JPEGImages'
mask_dir = '../data/VOCdevkit/VOC2012/SegmentationClass'
augmented_dir = '../data/VOCdevkit/VOC2012/AugmentedImages'

images, masks = load_images_and_masks(relation_folder, image_dir, mask_dir, augmented_dir)
print(f"Loaded {len(images)} images and {len(masks)} masks from {image_dir} and {augmented_dir} based on JSON files and augmented images.")


Loading augmented images and masks: 100%|██████████| 47/47 [00:00<00:00, 125.32it/s]


Loaded 2438 images and 2438 masks from ../data/VOCdevkit/VOC2012/JPEGImages and ../data/VOCdevkit/VOC2012/AugmentedImages based on JSON files and augmented images.
